In [ ]:

import numpy as np
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.transforms as mtrans
import matplotlib.patches as mpatches
import os

In [ ]:
# Preprocess data
sns.set_theme()
os.makedirs("plots", exist_ok=True)

res = pd.read_csv("results.csv")
#find best fortran implementation for each hw
hw = ["NVIDIA", "AMD", "INTEL"]
precision = ["SINGLE", "DOUBLE", "MIXED1", "MIXED2"]
use_case = ["LDC", "VKS"]
compilers = ["dpcpp", "AdaptiveCpp"]
best_impl = dict()
for u in use_case:
    best_impl[u] = dict()
    for h in hw:
        best_impl[u][h] = dict()
        for p in precision:
            filtered_df = res[((res["hw"] == h) & (res["usecase"] == u) & (res["impl"] == "fortran") & (res["precision"] == p))].sort_values(by="mlups").reset_index(drop=True)
            if not filtered_df.empty:
                first_mlups_value = filtered_df["mlups"].iloc[0]
                # Optionally, store the value in best_impl dictionary
                best_impl[u][h][p] = first_mlups_value
            else:
                best_impl[u][h][p] = None
    
print(best_impl)

# Get sycl values
sycl_csv = res[(res["impl"] == "sycl")]
# Divide sycl values by best fortran values
for u in use_case:
    for h in hw:
        for p in precision:
            rows = sycl_csv.loc[(sycl_csv["hw"] == h) & (sycl_csv["usecase"] == u) & (sycl_csv["precision"] == p)]
            if not rows.empty:
                sycl_csv.loc[(sycl_csv["hw"] == h) & (sycl_csv["usecase"] == u) & (sycl_csv["precision"] == p), "mlups"] = sycl_csv.loc[(sycl_csv["hw"] == h) & (sycl_csv["usecase"] == u) & (sycl_csv["precision"] == p), "mlups"].map(lambda x: x / best_impl[u][h][p]) 
# # Create the bar plot using catplot

for u in use_case:
    for h in hw:
        for p in precision:
            for c in compilers:
                rows = sycl_csv.loc[(sycl_csv["hw"] == h) & (sycl_csv["usecase"] == u) & (sycl_csv["precision"] == p) & (sycl_csv["parallelism"] == c)]
                if not rows.empty:
                    # Drop all rows except the the one with the hightest mlups
                    rows.sort_values(by="mlups", inplace=True)
                    sycl_csv.drop(rows.index[0:-1], inplace=True)
                    
#subsitute each "dpcpp" entry in column "compilers" to "Intel DPC++"
sycl_csv["parallelism"] = sycl_csv["parallelism"].str.replace("dpcpp", "Intel DPC++")

parallelisms = ["AdaptiveCpp", "Intel DPC++"]
total_plots = len(hw) * len(parallelisms)

fig, axes = plt.subplots(nrows=1, ncols=total_plots, figsize=(total_plots * 5, 5), sharey=True)
color_palette = sns.color_palette("viridis", len(precision))
color_map = dict(zip(precision, color_palette))

plot_index = 0
for parallelism in parallelisms:
    for h in hw:
        # Assuming 'use_case' and 'precision' are defined and 'res' is filtered accordingly
        for k, u in enumerate(use_case):
            for l, p in enumerate(precision):
                df_filtered = sycl_csv[(sycl_csv["hw"] == h) & (sycl_csv["usecase"] == u) & (sycl_csv["precision"] == p) & (sycl_csv["parallelism"] == parallelism)]
                if not df_filtered.empty:
                    mean_mlups = df_filtered["mlups"].mean()
                    ax = axes[plot_index]
                    ax.bar(k + l*0.2, mean_mlups, color=color_map[p], width=0.2, label=f'{p}' if plot_index == 0 else "")
                    # Modify the ax.bar to have just one hue for the precision
                    ax.set_title(f'{h}', y = -0.2)
                    if plot_index == 1 or plot_index == 4:
                        ax.set_xlabel(f'{parallelism}')
                        # move it to the top
                        ax.xaxis.set_label_position('top')
                    ax.axhline(1, color='red', linewidth=3, linestyle='--')
                    ax.set_xticks(range(len(use_case)))
                    ax.set_xticklabels(use_case),
                    # set hatch
                    if parallelism == "AdaptiveCpp":
                        # iterate over bars
                        for bar in ax.patches:
                            # set hatching
                            bar.set_hatch('/')
                            # set hatch color
                            bar.set_edgecolor('black')
                    else:
                        # iterate over bars
                        for bar in ax.patches:
                            # set hatching
                            bar.set_hatch('x')
                            # set hatch color
                            bar.set_edgecolor('black')
                    if plot_index == 0:
                        ax.set_ylabel('Speedup')
                    trans = mtrans.Affine2D().translate(6, 0)
                    for t in ax.get_xticklabels():
                        t.set_transform(t.get_transform()+trans)
                    # Increase all subplots font size
                    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
                                ax.get_xticklabels() + ax.get_yticklabels()):
                        item.set_fontsize(23)
                        
        plot_index += 1

plt.subplots_adjust(wspace=0, hspace=0)

legend_handles = [mpatches.Patch(color=color_map[p], label=p) for p in precision]

fig.legend(handles=legend_handles, title='Precision', loc='upper center', ncol=4, fontsize=20, title_fontsize=20, bbox_to_anchor=(0.5, 1.15))
# g.fig.suptitle('Bar Plot with Two Groups by Type', y=1.03) # Adjust title and its position
# g.set_axis_labels('Category', 'Value') # Set x and y axis labels
plt.savefig("plots/speedup.pdf", bbox_inches='tight')
plt.show()


sycl_csv = res[(res["impl"] == "sycl")]
for u in use_case:
    for h in hw:
        for p in precision:
            for c in compilers:
                rows = sycl_csv.loc[(sycl_csv["hw"] == h) & (sycl_csv["usecase"] == u) & (sycl_csv["precision"] == p) & (sycl_csv["parallelism"] == c)]
                if not rows.empty:
                    # Drop all rows except the the one with the hightest mlups
                    rows.sort_values(by="mlups", inplace=True)
                    sycl_csv.drop(rows.index[0:-1], inplace=True)
                    
#subsitute each "dpcpp" entry in column "compilers" to "Intel DPC++"
sycl_csv["parallelism"] = sycl_csv["parallelism"].str.replace("dpcpp", "Intel DPC++")

parallelisms = ["AdaptiveCpp", "Intel DPC++"]
total_plots = len(hw) * len(parallelisms)

fig, axes = plt.subplots(nrows=1, ncols=total_plots, figsize=(total_plots * 5, 5), sharey=True)
color_palette = sns.color_palette("viridis", len(precision))
color_map = dict(zip(precision, color_palette))

plot_index = 0
for parallelism in parallelisms:
    for h in hw:
        # Assuming 'use_case' and 'precision' are defined and 'res' is filtered accordingly
        for k, u in enumerate(use_case):
            for l, p in enumerate(precision):
                df_filtered = sycl_csv[(sycl_csv["hw"] == h) & (sycl_csv["usecase"] == u) & (sycl_csv["precision"] == p) & (sycl_csv["parallelism"] == parallelism)]
                if not df_filtered.empty:
                    mean_mlups = df_filtered["mlups"].mean()
                    ax = axes[plot_index]
                    ax.bar(k + l*0.2, mean_mlups, color=color_map[p], width=0.2, label=f'{p}' if plot_index == 0 else "")
                    # Modify the ax.bar to have just one hue for the precision
                    ax.set_title(f'{h}', y = -0.2)
                    if plot_index == 1 or plot_index == 4:
                        ax.set_xlabel(f'{parallelism}')
                        ax.xaxis.set_label_position('top')
                    ax.set_xticks(range(len(use_case)))
                    ax.set_xticklabels(use_case),
                    # set hatch
                    if parallelism == "AdaptiveCpp":
                        # iterate over bars
                        for bar in ax.patches:
                            # set hatching
                            bar.set_hatch('/')
                            # set hatch color
                            bar.set_edgecolor('black')
                    else:
                        # iterate over bars
                        for bar in ax.patches:
                            # set hatching
                            bar.set_hatch('x')
                            # set hatch color
                            bar.set_edgecolor('black')
                    if plot_index == 0:
                        ax.set_ylabel('MLUP/s')
                    trans = mtrans.Affine2D().translate(6, 0)
                    for t in ax.get_xticklabels():
                        t.set_transform(t.get_transform()+trans)
                    # Increase all subplots font size
                    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
                                ax.get_xticklabels() + ax.get_yticklabels()):
                        item.set_fontsize(23)
                        
        plot_index += 1

plt.subplots_adjust(wspace=0, hspace=0)

legend_handles = [mpatches.Patch(color=color_map[p], label=p) for p in precision]

fig.legend(handles=legend_handles, title='Precision', loc='upper center', ncol=4, fontsize=20, title_fontsize=20, bbox_to_anchor=(0.5, 1.15))
# g.fig.suptitle('Bar Plot with Two Groups by Type', y=1.03) # Adjust title and its position
# g.set_axis_labels('Category', 'Value') # Set x and y axis labels
plt.savefig("plots/mlups.pdf", bbox_inches='tight')
plt.show()

In [ ]:
from matplotlib.colors import LinearSegmentedColormap


sycl_csv = res[(res["impl"] == "sycl")] # Reset csv
# add new coloumn to sycl_csv eith the combination of values in columns "alloc_type" and "range"
sycl_csv["alloc_type_range"] = sycl_csv["alloc_type"] + "\n" + sycl_csv["range"]
fortran_par_types = ["doconcurrent", "openacc", "offload"]         

# duplcate each row three times and add the values in fortran_par_types to the new column "fortran_par"
sycl_csv = sycl_csv.loc[sycl_csv.index.repeat(3)].reset_index(drop=True)
sycl_csv["fortran_par"] = fortran_par_types * int(len(sycl_csv) / 3)
sycl_csv["speedup"] = 0

for h in hw:
    for p in precision:
        for u in use_case:
            for fortran_par in fortran_par_types:
#                 # print(f"hw: {h}, precision: {p}, usecase: {u}, fortran_par: {fortran_par}")
                value = res[(res["impl"] == "fortran") & (res["parallelism"] == fortran_par) & (res["hw"] == h) & (res["usecase"] == u) & (res["precision"] == p)].reset_index(drop=True)
                if not value.empty:
                    # value["mlups"].iloc[0]
                    subset = sycl_csv.loc[(sycl_csv["hw"] == h) & (sycl_csv["usecase"] == u) & (sycl_csv["precision"] == p) & (sycl_csv["fortran_par"] == fortran_par)]
                    subset["speedup"] = subset["mlups"] / value["mlups"].iloc[0]
                    sycl_csv.update(subset)
                # elif h == "INTEL" and fortran_par == "doconcurrent":
                #     subset = sycl_csv.loc[(sycl_csv["hw"] == h) & (sycl_csv["usecase"] == u) & (sycl_csv["precision"] == p) & (sycl_csv["fortran_par"] == fortran_par)]
                #     subset["speedup"] = np.max(subset["speedup"])
                #     sycl_csv.update(subset)

for p in precision:
    for u in use_case:
        # get all value with hw intel, precision p, usecase u
        sycl_csv_tmp = sycl_csv[(sycl_csv["hw"] == "INTEL") & (sycl_csv["precision"] == p) & (sycl_csv["usecase"] == u)]
        max_idx = sycl_csv_tmp["speedup"].idxmax()
        max_value = sycl_csv_tmp.loc[max_idx, "speedup"]
        subset = sycl_csv.loc[(sycl_csv["hw"] == "INTEL") & (sycl_csv["precision"] == p) & (sycl_csv["usecase"] == u) & (sycl_csv["fortran_par"] == "doconcurrent")]
        subset["speedup"] = max_value
        sycl_csv.update(subset)

        
# # make seaborn heatmap
# Filter only LDC use case
sycl_csv = sycl_csv[sycl_csv["usecase"] == "LDC"]
beaufity_precision = dict(
    SINGLE="Single Precision",
    DOUBLE="Double Precision",
    MIXED1="Mixed Precision 1",
    MIXED2="Mixed Precision 2"
)

# renane each "doconcurrent" entry from column "fortran_par" to "DC"
sycl_csv["fortran_par"] = sycl_csv["fortran_par"].str.replace("doconcurrent", "DC")

for h in hw:
    for compiler in compilers:
        sycl_csv_tmp = sycl_csv[(sycl_csv["hw"] == h) & (sycl_csv["parallelism"] == compiler)]
        # ax = sns.catplot(kind="bar", x='fortran_par', y='speedup', hue='alloc_type_range', col="precision", data=sycl_csv_tmp)
        # ax.set_titles('{col_name}')
        # ax.set_axis_labels('Fortran Parallelism', 'Speedup')
        # ax.fig.suptitle(f'{h} - {compiler}')
        # plt.show()
        # sycl_csv_tmp.to_csv(f"{h}_{compiler}.csv", index=False)
        for p in ["MIXED1", "MIXED2"]:
            sycl_csv_tmp_2 = sycl_csv_tmp[sycl_csv_tmp["precision"] == p]
            agg_sycl_csv = sycl_csv_tmp_2.groupby(['alloc_type_range', 'fortran_par'])['speedup'].mean().reset_index()

            # Pivot the aggregated DataFrame
            pivot_sycl_csv = agg_sycl_csv.pivot(index='alloc_type_range', columns='fortran_par', values='speedup')

            # Plot the heatmap
            # Define the colors for the colormap (red, yellow, green)
            colors = [(0.7, 0, 0), (0.7, 0.7, 0), (0, 0.5, 0)]  # Example darker shades
        # Create the colormap with darker colors
            cmap = LinearSegmentedColormap.from_list("DarkRedYellowGreen", colors)
            # make a colormap for the heatmap that goes from red to yellow to green
            
            ax = sns.heatmap(data=pivot_sycl_csv, annot=True,fmt=".1f",cmap=cmap, linewidth=0.5)
            ax.set_title(f'{beaufity_precision[p]}')
            _xlabels = ax.get_xticklabels()
            for labels in _xlabels:
                labels.set(text=labels.get_text().replace(' ',"\n"))
            ax.set_xticklabels(_xlabels, rotation=-360, ha='center', rotation_mode='anchor')
            ax.set_yticklabels(ax.get_yticklabels(), ha='center', x=-0.17, rotation=0)
            ax.set(xlabel='',ylabel='')
            # set the annotated values to be in the center of the cells
            for t in ax.texts:
                if h == "INTEL" and t.get_position()[0] == 0.5:
                    # set the unicode symbol for infinity
                    t.set_text("N.D.")
                    # t.set_text("Infinite")
                elif t.get_text() == "0.0":
                        t.set_text("N.S.")
                else:
                    t.set_text(t.get_text() + "x")
                # Increase font size
                t.set_fontsize(23)
            if p == "DOUBLE" or p == "MIXED2":
                # make the ytickslaels white
                # ax.set_yticklabels(ax.get_yticklabels(), color='white')
                ax.set_yticklabels([])
            # Increase font size
            for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
                        ax.get_xticklabels() + ax.get_yticklabels()):
                item.set_fontsize(23)
            

            plt.savefig(f"plots/{h}_{compiler}_{p}.pdf", bbox_inches='tight')
            plt.show()

# ax = sns.heatmap(data = sycl_csv)


# for fortran_par in fortran_par_types:
#     value = res[(res["impl"] == "fortran") & (res["parallelism"] == fortran_par)]
#     sycl_csv[fortran_par] = sycl_csv["mlups"] / 


In [ ]:
## barplot with speedup compared to offload
from matplotlib.colors import LinearSegmentedColormap


sycl_csv = res[(res["impl"] == "sycl")] # Reset csv
# add new coloumn to sycl_csv eith the combination of values in columns "alloc_type" and "range"
sycl_csv["alloc_type_range"] = sycl_csv["alloc_type"] + "\n" + sycl_csv["range"]
sycl_csv["speedup"] = 0

for h in hw:
    for p in precision:
        for u in use_case:
#                 # print(f"hw: {h}, precision: {p}, usecase: {u}, fortran_par: {fortran_par}")
            value = res[(res["impl"] == "fortran") & (res["parallelism"] == "offload") & (res["hw"] == h) & (res["usecase"] == u) & (res["precision"] == p)].reset_index(drop=True)
            if not value.empty:
                # value["mlups"].iloc[0]
                subset = sycl_csv.loc[(sycl_csv["hw"] == h) & (sycl_csv["usecase"] == u) & (sycl_csv["precision"] == p)]
                subset["speedup"] = subset["mlups"] / value["mlups"].iloc[0]
                sycl_csv.update(subset)
        
# # make seaborn heatmap
# Filter only LDC use case
sycl_csv = sycl_csv[sycl_csv["usecase"] == "LDC"]
beaufity_precision = dict(
    SINGLE="Single Precision",
    DOUBLE="Double Precision",
    MIXED1="Mixed Precision 1",
    MIXED2="Mixed Precision 2"
)

# fig = sns.relplot(kind="scatter", x='alloc_type_range', y='speedup', hue='hw', style='precision', col="parallelism", data=sycl_csv, s=200)
# Set hue color to red, blue, and green for NVIDIA, AMD, and INTEL respectively
sycl_csv["parallelism"] = sycl_csv["parallelism"].str.replace("dpcpp", "Intel DPC++")

fig = sns.relplot(kind="scatter", x='alloc_type_range', y='speedup', hue='hw', style='precision', col="parallelism", data=sycl_csv, s=200, palette={'NVIDIA': 'Green', 'AMD': 'Brown', 'INTEL': 'Blue'})
# add a red line at y=1
fig.map(plt.axhline, y=1, color='red', linestyle='--')
fig.set_titles('{col_name}')
fig.set_xlabels("")
fig.set_ylabels("Speedup over FORTRAN Offload")

plt.savefig("plots/speedup_over_offload.pdf", bbox_inches='tight')
